## How to add Realtime Indicators to Crypto Market Data in Python

This tutorial will show you how to add indicators to cryptocurrency market data in Python. Indicators are statistics of price/volume data used to measure current market conditions as well as to forecast trends. Basically, they generate trading signals and are a piece of the broader trading strategy that also involves risk and portfolio management. Common indicators are the [RSI](https://www.investopedia.com/terms/r/rsi.asp) and [MACD](https://www.investopedia.com/terms/m/macd.asp)

**What we will accomplish:**
1. Create our first indicator (SMA) and plot the results.
2. Use LiveDataFrame to do this in realtime, across many coins
3. Filter for new oppurtunities
4. Create a Momentum Strategy using the RSI indicator

## The Hello World of Indicators

The Simple Moving Average (SMA) strategy is popular among technical analysts. The strategy is very simple:
- If the <font color='red'>shorter SMA</font> is _higher_ in value than the <font color='green'>longer SMA</font>
    - Go long 
- If the <font color='red'>shorter SMA</font> is _lower_ in value than the <font color='green'>longer SMA</font>
    - Go Short (or Neutral)
        
**In picture form:** _Go Short on Mar 14, Go Long on April 16, Short again on May 14_
<img src="sma_picture.png">

## Lets go ahead and create it for a single coin!

For the first example, we are going to load a LiveDataFrame for `ETHUSD`. We will then add a short moving average and a long moving average indicator with the candles and then plot the results. It's free to [sign up](`app.livedataframe.com/users/new`) and you get your API keys immediately. Lets go ahead and start a LiveDataFrame.

In [ ]:
from livedataframe import LiveExchange # The live data

livedf = LiveExchange(
    public_key='<YOUR-PUBLIC-KEY>', # Enter your public key that was emailed
    secret_key='<YOUR-PRIVATE-KEY>', # Enter your secret key that was emailed
    exchange = 'bitfinex', # Enter your exchange
    symbols = ['ETHUSD'], # Our symbol
    lookback_period = '4H') # How much historical data we want

# Start it up!
livedf.start()

We can easily view the coins that we have requested by using the `symbols` dictionary. Let's see what coins are loaded and live updating.

In [ ]:
livedf.symbols.keys()

In [ ]:
# Lets look at the data for ETHUSD
livedf.symbols['ETHUSD'].tail()

Perfect, it looks like ETHUSD is ready!

### Plotting the price

In [ ]:
%matplotlib inline
# %matplotlib inline is a "magic function" to make our lives easier. 
# You can see many more at: https://ipython.readthedocs.io/en/stable/interactive/magics.html

import seaborn as sns; sns.set() # Pretty plots

livedf.symbols['ETHUSD']['last_price'].plot()

### Plotting the price in realtime

If we use a while loop to plot the data after every new update (5 seconds) then we will be able to see the plot in real time!

We can also specify that we are only interested in more recent data. Lets write the code that will display a live plot of the last 5 minutes of data.

In [ ]:
%matplotlib inline
import time # For adding a 5 second delay in between getting new price updates
import matplotlib.pyplot as plt # Import matplotlib
import seaborn as sns; sns.set() # Pretty charts (Optional)
from IPython.display import display, clear_output # So that we can refresh our Jupyter cell output

while True:
    
    # We are only interested in the last 5 minutes for viewing the plot
    last_hour = livedf.symbols['ETHUSD']['last_price'].last('5T')
    
    # Generate our plot of last_price data for ETHUSD
    # As you can see, we can also give plots a title and choose sizes to our liking.
    last_hour.plot(figsize=(15,6), title='ETHUSD - LAST PRICE')
    
    # Wait until there is a new plot to show before clearing the old one
    clear_output(wait=True)
    # Show the new plot
    plt.show()
    
    # Wait 5 seconds
    time.sleep(5)

### Add our SMA Indicators

The _average_ price is taken inside of a window containing some price points. Then the window _moves_ to the next few. Then the window _moves_ to the next few. We are _smoothing_ out the price by taking the moving average. The larger the window, the smoother and slower the average will move.

Pandas has a great function that will automatically do this calculation. Use the rolling (_moving_) method to go through the price points and apply the mean (_average_).

In [ ]:
# Lets make a copy of our ETHUSD prices, and resample it to have a one minute bar size. 
ethusd_df = livedf.symbols['ETHUSD'][['last_price']].resample('T', label='right').last().ffill()

# Create new columns on our ETHUSD prices dataframe: 

# We want our short sma window to be 5 bars long
ethusd_df['short_sma'] = ethusd_df['last_price'].rolling(5).mean()
# We want our long sma to be 20 bars long
ethusd_df['long_sma'] = ethusd_df['last_price'].rolling(20).mean()

# Lets take a look
ethusd_df.head(5)

If we look at the beginning of our prices using the _head_ method, we can see a few `NaN` rows. We should expect these when calculating indicators. For our short sma window, we should expect 5 `NaN` rows. How do we know this? Because the indicator needs at LEAST 5 previous price points to calculate the average. For the same reason, the long window will have 20 `NaN` rows. Don't worry, we can easily get rid of them.

In [ ]:
# ByeBye NaN's 
ethusd_df = ethusd_df.dropna()

# Lets take a look
ethusd_df.head(5)

### Plot our Indicators

Now that we have our indicators calculated, lets plot them.

In [ ]:
ethusd_df[['last_price', 'short_sma', 'long_sma']].plot(figsize=(15,6), title='ETHUSD - CLOSING PRICE')

### Plotting the indicators in realtime

Static plots are pretty boring right? Lets watch the indicators in realtime so we can spot the oppurtunities to go long!

In [ ]:
%matplotlib inline
import time 
import matplotlib.pyplot as plt 
import seaborn as sns; sns.set()
from IPython.display import display, clear_output 

while True:
    
    # Copy the resampled ETHUSD dataframe
    ethusd_df = livedf.symbols['ETHUSD'][['last_price']].resample('T', label='right').last().ffill()
    
    # Like before, we want our short sma window to be 5 periods long & long sma window 20 periods long
    ethusd_df['short_sma'] = ethusd_df['last_price'].rolling(5).mean()
    ethusd_df['long_sma'] = ethusd_df['last_price'].rolling(20).mean()
    
    # Generate our plot for ETHUSD. 
    # We want the 'last_price','short_sma', and 'long_sma' columns to be plotted
    # As you can see, we can also give plots a title and choose sizes to our liking.
    ethusd_df[['last_price', 'short_sma', 'long_sma']].last('H').plot(figsize=(15,6), title='ETHUSD - LAST PRICE')
    
    # Wait until there is a new plot to show before clearing the old one
    clear_output(wait=True)
    # Show the new plot
    plt.show()
    
    # Wait 5 seconds
    time.sleep(5)

## Use LiveDataFrame to do this in realtime, across many coins

Say we are interested in looking for oppurtunities to buy a coin. These oppurtunities exist when the short SMA is greater than the long SMA.

So far we have focused on one coin, ETHUSD. We can use LiveDataFrame to monitor **every coin** on an exchange and alert us when a new oppurtunity comes.

Lets stop our LiveDataFrame from earlier so we can grab more coins that just ETHUSD.

In [ ]:
livedf.stop()

### Looking for oppurtunities on Bitfinex

We will use one minute bar sizes. Our short SMA will be 5 periods and long SMA 20 periods. Therefore, we need at least (5 min x 5 periods) 25 minutes of data for our short SMA, and (5 min x 20 periods) 100 minutes of data for our long SMA.

Bitfinex has a lot of coins. For the time being, lets just look at the coins that trade with USD. 

In [ ]:
from livedataframe import ExchangeInfo # Information about what exchanges and coins are available.
from livedataframe import LiveExchange # The live data

# Grab every coin available on Bitfinex
all_symbols_bitfinex = ExchangeInfo.list_symbols_for('bitfinex')

# Since we are only interested in USD pairs, we will filter our list down to those.
usd_symbols = [symbol for symbol in all_symbols_bitfinex if 'USD' in symbol]

livedf = LiveExchange(
    public_key='<YOUR-PUBLIC-KEY>', # Enter your public key that was emailed
    secret_key='<YOUR-PRIVATE-KEY>', # Enter your secret key that was emailed
    exchange = 'bitfinex', # Enter your exchange
    symbols = usd_symbols, # Our USD symbols
    lookback_period = '105M') # We need 100min of data. We'll grab 105 Minutes to be safe.

# Start it up!
livedf.start()

In [ ]:
# Lets see all the coins we have access too..
print(livedf.symbols.keys())
print("\nWe have {} symbols to monitor that trade in USD".format(len(livedf.symbols.keys())))

### Add the SMA indicators to every coin

Just like before, we will use a while loop to get the latest data for our coins as it arrives. This time, we will go through every single coin and apply our indicators. I have fairly thick glasses, but even I would have a hard time monitoring 70 different charts. For our strategy, we just need to know if the short SMA is greater than the long SMA. We don't really need the charts. We just need what the values are, _right now_.

First, we will add the indicators, and store the results. Then we will create a view for the results by creating a new dataframe. Finally, we will make sure it updates in real time.

In [ ]:
%matplotlib inline
import time 
import matplotlib.pyplot as plt 
import seaborn as sns; sns.set()
from IPython.display import display, clear_output 
import pandas as pd

while True:
    
    # Create a DataFrame to hold our results
    view_df = pd.DataFrame()
    
    # For every symbol we are monitoring, add our indicators.
    for symbol, ldf in livedf.symbols.items():
         
        # Copy the resampled dataframe
        temp_ldf = ldf[['last_price']].resample('T', label='right').last().ffill()

        # Short sma window 5 periods & long sma window 20 periods
        temp_ldf['short_sma'] = temp_ldf['last_price'].rolling(5).mean()
        temp_ldf['long_sma'] = temp_ldf['last_price'].rolling(20).mean()
        
        # If short sma is greater than long sma, then we have a buy signal!
        # Use iloc[-1] to get the most recent value
        if temp_ldf['short_sma'].iloc[-1] > temp_ldf['long_sma'].iloc[-1]:
            
            temp_ldf['buy_signal'] = True
            
        else:
            
            temp_ldf['buy_signal'] = False
            
        # We only need the MOST RECENT price & sma values to create our view
        temp_ldf = temp_ldf.tail(1)
        
        # We will index it by name for convenience.
        temp_ldf.index = [symbol]
        
        # Then, we store this result in a dataframe to view later. 
        view_df = view_df.append(temp_ldf)
    
    # View the result
    clear_output(wait=True)
    display(view_df)
    
    # Wait 5 seconds, then do it all over again
    time.sleep(5)

### Making it Better

There is a lot of information still being displayed that we don't care about. We only want to see the buy oppurtunities, and ideally, we want to know how _fresh_ this oppurtunity is. We can determine how fresh our code is by going back in time, one bar at a time and finding out when the short sma FIRST crossed over the long sma. 

To make this view better, we will:
- Determine how fresh the oppurtunity is
- Filter out all coins that don't fit our criteria

Let's rework our code a little bit.

In [ ]:
%matplotlib inline
import time 
import matplotlib.pyplot as plt 
import seaborn as sns; sns.set()
from IPython.display import display, clear_output 
import pandas as pd

while True:

    view_df = pd.DataFrame()
    
    for symbol, ldf in livedf.symbols.items():
         
        temp_ldf = ldf[['last_price']].resample('T', label='right').last()

        temp_ldf['short_sma'] = temp_ldf['last_price'].rolling(5).mean()
        temp_ldf['long_sma'] = temp_ldf['last_price'].rolling(20).mean()
        
        # This is just a faster way of checking every row if short_sma is greater than long_sma
        temp_ldf['buy_signal'] = temp_ldf['short_sma'] > temp_ldf['long_sma']
            
        # This time, if our signal is a buy...
        if temp_ldf['buy_signal'].iloc[-1] == True:
            
            # Locate all the times the signal was NOT a buy..
            # Grab the last time that this was not a buy signal
            last_neutral_time = temp_ldf.loc[temp_ldf['buy_signal'] == False].index[-1]
            
            # Now grab the most recent time
            current_time = temp_ldf.index[-1]
            
            # How long has this buy signal existed?
            temp_ldf['freshness'] = str(current_time - last_neutral_time)
            
            # We've already calculated our indicators, signal, and freshness. 
            # Now we do the same as before by storing the result and creating a view
            temp_ldf = temp_ldf.tail(1)
            temp_ldf.index = [symbol]
            view_df = view_df.append(temp_ldf)
            
    
    clear_output(wait=True)
    display(view_df.sort_values('freshness'))
    
    time.sleep(5)

### BONUS: Get to the exchange as soon as you see an oppurtunity!

We can add a link our strategy view so that you can get to the oppurtunity as SOON as you see it!

In [ ]:
# create a function to format our link so it is clickable
def make_clickable(symbol):
    
    first_part, second_part = symbol[:3], symbol[3:]
    url = 'https://www.bitfinex.com/t/'+first_part+':'+second_part

    return '<a target="_blank" href="{}">{}</a>'.format(url, symbol)

In [ ]:
%matplotlib inline
import time 
import matplotlib.pyplot as plt 
import seaborn as sns; sns.set()
from IPython.display import display, clear_output 
import pandas as pd

while True:

    view_df = pd.DataFrame()
    
    for symbol, ldf in livedf.symbols.items():
         
        temp_ldf = ldf[['last_price']].resample('T', label='right').last()

        temp_ldf['short_sma'] = temp_ldf['last_price'].rolling(5).mean()
        temp_ldf['long_sma'] = temp_ldf['last_price'].rolling(20).mean()
        
        temp_ldf['buy_signal'] = temp_ldf['short_sma'] > temp_ldf['long_sma']
            
        if temp_ldf['buy_signal'].iloc[-1] == True:
            
            last_neutral_time = temp_ldf.loc[temp_ldf['buy_signal'] == False].index[-1]
            current_time = temp_ldf.index[-1]
            
            temp_ldf['freshness'] = str(current_time - last_neutral_time)
            
            # Store result as before
            temp_ldf = temp_ldf.tail(1)
            temp_ldf.index = [symbol]
            
            # Create a column for the link
            temp_ldf['link'] = symbol
            
            view_df = view_df.append(temp_ldf)
            
    
    clear_output(wait=True)
    # Prepare a special view
    view_df = view_df.sort_values('freshness')
    display(view_df.style.format(make_clickable, subset=['link']))
    
    time.sleep(5)

## Next Steps

Congratulations! you've just learned how add realtime indicators to monitor many symbols across an exchange. 

Here are some things you can try next:
- Convert this script into a function that you can call with parameters:
    - To change the bar size
    - To add filtering for how fresh a new signal should be
    - To change the length of short sma and long sma
- Explore other indicators and strategies.
- Check out the [stockstats](https://github.com/jealous/stockstats) python library for a huge range of free indicators.
- Make your own indicator from the data provided by LiveDataFrame.

If you have questions, feel free to reach out to `support@livedataframe.com`, and we'll be happy to help you out! 

_DISCLAIMER: The above references an opinion and is for information purposes only. It is not intended to be investment advice. Seek a duly licensed professional for investment advice._